GPT : 기존에 없는 새로운 문장을 만들어내기 위해 사용한 모델입니다. 기존에 개발자가 입력해둔 문장을 기계적으로 출력하는데 그치지 않기때문에 보다 생동감 있는 대화를 기대할 수 있습니다. 
'데이터의 연속성'이 필요한 부분에서 GPT를 사용합니다. 질문자가 입력한 정보를 챗봇이 기억하고 있어야 맥락에 맞는 대화를 할 수 있습니다. 질문자에게 부적절한 응답을 하는 오류를 줄이기 위해서는 질문자가 직전에 입력한 내용 뿐만아니라 이전의 대화 내역도 함께 재입력해주는 작업이 필요했습니다. GPT모델은 연속적인 대화 뒤에 이어질 적절한 문장을 출력하기 위해 사용했습니다.

데이터는 각각의 모델에 맞는 적합한 형태로 만들기 위해서 개별적인 전처리를 수행했고, all_chat.csv는 GPT학습에 적합하도록 전처리한 파일입니다.





## google 드라이버 연결

In [ ]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive/')
os.chdir('drive/My Drive/open_directory/test/dataset')
!pwd

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/18A-45l2iGPLTzGSEx1Bd0wrE_qi8y2Ai/open_directory/test/dataset


## import

In [ ]:
import pandas as pd
import numpy as np

## 0) 변수설정

변수명 
- 드랍할 컬럼 리스트 
  - drop_col = []
- 라벨링할 컬럼 리스트 
  - labeling_cols = []
- csv로 저장할 파일명.csv확장자까지 
  - filecsv = "all_chat.csv"
- gpt모델에 넣을 데이터 생성시, 열 사이에 tap을 넣어 문장을 구분할 수 있게 합니다.

In [ ]:
labeling_col =['연령', '성별', '상황키워드', '감정_대분류',	'감정_소분류']  
drop_col=['번호', '신체질환'] # 지우고 싶은 컬럼(라벨링 안할 것들)
filecsv = "all_chat.csv"

In [ ]:
# gpt모델에 넣을 데이터 생성시, 열 사이 에 tap으로 합치기
btw_col = "[SEP]"

# gpt모델 데이터의 파일명
filecsv2 = "gpt_train_data.csv"
filecsv3 = "gpt_val_data.csv"

In [ ]:
# 컬럼지정 변수 개수 안맞을 때 오류나서 실행 멈추게
while (len(drop_col)+len(labeling_col) != 7):
  print(개수안맞음)

## 1) 데이터 다운로드

In [ ]:
train_data = pd.read_excel('training.xlsx')
val_data = pd.read_excel('validation.xlsx')
all_df = pd.concat([train_data, val_data]).iloc[:, 1:]

## 2) 데이터 탐색

In [ ]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46009 entries, 0 to 5129
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연령      46009 non-null  object
 1   성별      46009 non-null  object
 2   상황키워드   46009 non-null  object
 3   신체질환    46009 non-null  object
 4   감정_대분류  46009 non-null  object
 5   감정_소분류  46009 non-null  object
 6   사람문장1   46009 non-null  object
 7   시스템응답1  46009 non-null  object
 8   사람문장2   46009 non-null  object
 9   시스템응답2  46009 non-null  object
 10  사람문장3   34933 non-null  object
 11  시스템응답3  34933 non-null  object
 12  사람문장4   1623 non-null   object
 13  시스템응답4  1498 non-null   object
dtypes: object(14)
memory usage: 5.3+ MB


## 3) 데이터 전처리

### 공백 전처리

In [ ]:
# 라벨링 전 전처리
all_df2=all_df.apply(lambda x: x.str.strip(), axis = 1)

all_df2[['연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류']].nunique()

연령         4
성별         2
상황키워드     12
신체질환       3
감정_대분류     6
감정_소분류    58
dtype: int64

### 대화 Nan처리

In [ ]:
# 발화만있고 응답이 없거나 발화는 없는데 응답만 있는 대화쌍의 조건
Nan_cond = all_df2['사람문장4'].isnull() != all_df2['시스템응답4'].isnull()

# 조건이 해당되는 사람문장4를 Nan처리
all_df2.loc[(Nan_cond),'사람문장4'] = np.nan
all_df2.loc[(Nan_cond),'시스템응답4'] = np.nan

In [ ]:
all_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46009 entries, 0 to 5129
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연령      46009 non-null  object
 1   성별      46009 non-null  object
 2   상황키워드   46009 non-null  object
 3   신체질환    46009 non-null  object
 4   감정_대분류  46009 non-null  object
 5   감정_소분류  46009 non-null  object
 6   사람문장1   46009 non-null  object
 7   시스템응답1  46009 non-null  object
 8   사람문장2   46009 non-null  object
 9   시스템응답2  46009 non-null  object
 10  사람문장3   34933 non-null  object
 11  시스템응답3  34933 non-null  object
 12  사람문장4   1497 non-null   object
 13  시스템응답4  1497 non-null   object
dtypes: object(14)
memory usage: 5.3+ MB


### columns drop

In [ ]:
all_df2.columns

Index(['연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1', '시스템응답1',
       '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'],
      dtype='object')

In [ ]:
for col in drop_col:
    if col in all_df2.columns:
      all_df2 = all_df2.drop(columns=col)
      print(f'{col}제거했습니다')
    else : 
      print('이미 제거 됨')
all_df2.head()

이미 제거 됨
신체질환제거했습니다


,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,청년,남성,"연애, 결혼, 출산",기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,노년,남성,"건강, 죽음",불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,청소년,여성,학업 및 진로,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,노년,남성,재정,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,노년,여성,재정,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


### 라벨링

In [ ]:
for label in labeling_col:
  temp_list = []
  temp_list = all_df2[label].unique().tolist()  # 임시 리스트에 고유값 넣기
  all_df2[label] = all_df2[label].map(lambda x :temp_list.index(x)) # 정수화

In [ ]:
# 라벨링 자동 되는 모습.
all_df2.head()

,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,0,0,0,0,0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,1,0,1,1,1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,2,1,2,2,2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,1,0,3,0,0,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,1,1,3,0,3,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


### 대화 합치고 연속성 살리기


챗봇용 학습 데이터가 아래와 같이 존재할 때:

|질1|답1|질2|답2|질3|답3|질4|답4|
|---|---|---|---|---|---|---|---|
|Q11|A11|Q12|A12|Q13|A13|Q14|A14|
|Q21|A21|Q22|A22|Q23|A23|Q24|A24|

아래와 같이 학습데이터를 구성한다:

|질|답|
|---|---|
|Q11|A11|
|Q11 A11 Q12|A12|
|Q11 A11 Q12 A12 Q13|A13|
|Q11 A11 Q12 A12 Q13 A13 Q14|A14|
|Q21|A21|
|Q21 A21 Q22|A22|
|Q21 A21 Q22 A22 Q23|A23|
|Q21 A21 Q22 A22 Q23 A23 Q24|A24|


In [ ]:
# 열(대화들)을  조건에 맞게 결합

#사람문장2열위치=[-6:] 에 # Q1 \t A1 \t Q2 
condi1 = all_df2['사람문장1'] + btw_col + all_df2['시스템응답1'] + btw_col + \
all_df2['사람문장2'] 

#사람문장3열위치=[-4:] 에 # Q1 \t A1 \t Q2 \t A2 \t Q3
condi2 = all_df2['사람문장1'] + btw_col + all_df2['시스템응답1'] + btw_col + \
all_df2['사람문장2']  + btw_col + all_df2['시스템응답2'] + btw_col + \
all_df2['사람문장3']

#사람문장4열위치=[-2:] 에 # Q1 \t A1 \t Q2 \t A2 \t Q3 \t A3 \t Q4
condi3 = all_df2['사람문장1'] + btw_col + all_df2['시스템응답1'] + btw_col + \
all_df2['사람문장2']  + btw_col + all_df2['시스템응답2'] + btw_col + \
all_df2['사람문장3']+ btw_col + all_df2['시스템응답3'] + btw_col + all_df2['사람문장4']

In [ ]:
df_gpt = all_df2.copy()

In [ ]:
# insert
df_gpt.insert((len(df_gpt.columns)-6), 'Q1_A1_Q2', condi1)
df_gpt.insert((len(df_gpt.columns)-4), 'Q1_A1_Q2_A2_Q3', condi2)
df_gpt.insert((len(df_gpt.columns)-2), 'Q1_A1_Q2_A2_Q3_A3_Q4', condi3)

In [ ]:
# 남아있는 기존 열들 drop
df_gpt.drop(['사람문장2', '사람문장3', '사람문장4'], axis=1, inplace=True)

In [ ]:
# 문장 쌍으로 만든 리스트
qa_cols=[]
qa_cols.append(df_gpt.columns[-8:-6].tolist())
qa_cols.append(df_gpt.columns[-6:-4].tolist())
qa_cols.append(df_gpt.columns[-4:-2].tolist())
qa_cols.append(df_gpt.columns[-2:].tolist())
qa_cols

[['사람문장1', '시스템응답1'],
 ['Q1_A1_Q2', '시스템응답2'],
 ['Q1_A1_Q2_A2_Q3', '시스템응답3'],
 ['Q1_A1_Q2_A2_Q3_A3_Q4', '시스템응답4']]

In [ ]:

# eh_col = [] # 각 문장별 컬럼값들이들어갈 리스트
in_list = [] # ['사람문장1', '시스템응답1']각각들어가는

# 바뀔 컬럼명 리스트 ['Q', 'A',~~~]
new_col = ['Q','A']
new_col.extend(labeling_col)

df2 = pd.DataFrame()

for qanswer in qa_cols:
  in_list = qanswer

  # 컬럼명리스트 작성 
  eh_col = []
  for tmp in in_list:
    eh_col.append(tmp)
  eh_col.extend(labeling_col)

  # 작성된 컬럼명의 df을 추출해 df_n 생성 후 new_col으로 컬럼이름 다시 바꾸기
  df_n = df_gpt[eh_col]
  df_n.columns = new_col

  # df에 df_n추가
  df2 = df2.append(df_n)

# 대화4의 Nan값들 제거
df2.dropna(axis=0,  inplace = True)
df2.reset_index(inplace=True)

In [ ]:
pd.to_csv(df2,filecsv)
df2.tail()

,Q,A,연령,성별,상황키워드,감정_대분류,감정_소분류
128442,쉬는 시간에 선생님 몰래 같은 반 애들이 친구를 때리는 걸 봐서 화가 났어.[SEP...,담임 선생님께 말씀 드리면 가해 학생들이 징계를 받게 될 수 있군요.,2,0,6,4,42
128443,내가 좋아하는 일을 해야 할지 부모님이 원하는 일을 해야 할지 알 수 없어서 혼란스...,이 일에 대한 나의 열정을 말씀 드리려 하는군요.,2,1,2,2,18
128444,친구와 함께 가기로 한 여행을 나 혼자 가게 되었어. 조금 당황스러워.[SEP]함께...,오랜만에 고향에 가셔서 마음껏 즐기고 오셨으면 좋겠어요.,1,1,7,2,38
128445,친구들과 싸움이 없는 날이 하루도 없네.[SEP]계속 싸운다니 마음이 암담하시겠어요...,마음을 잘 다스려서 친구들과 잘 지냈으면 좋겠어요.,2,0,6,3,5
128446,요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나.[SEP]...,가족들과 진중한 대화를 나누고 관계가 나아지기를 바라요.,3,0,11,4,52
